In [1]:
! pip install voila
! jupyter serverextension enable --sys-prefix voila
! jupyter serverextension enable --sys-prefix voila.extension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter command `jupyter-serverextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
     

In [2]:
! pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [3]:
import numpy as np
import pandas as pd
import os
import torch
import torchvision
import torch.nn as nn
import torchvision.utils
import matplotlib.pyplot as plt
import seaborn as sns
#import cv2
from sklearn import tree, ensemble
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import load_iris
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch.autograd import Variable
import torchvision.utils
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch.optim as optim
from ipywidgets import interactive, widgets
from IPython.core.display import display, HTML
from IPython.display import display

/var/folders/r_/14gdjhxj3zg_5mkbmbpnb78c0000gn/T/ipykernel_18261/497124971.py:33: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
class AgeImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        imputer = SimpleImputer(strategy="median")
        X['Age'] = imputer.fit_transform(X[['Age']])
        return X

In [5]:
class FeatureEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        encoder = OneHotEncoder()
        matrix = encoder.fit_transform(X[['Sex']]).toarray()
        column_names = ["Female", "Male"]
        for i in range(len(column_names)):
            X[column_names[i]] = matrix.T[i]
        matrix = encoder.fit_transform(X[['Embarked']]).toarray()
        column_names = ["France", "Ireland", "Southham"]
        for i in range(len(column_names)):
            X[column_names[i]] = matrix.T[i]
        matrix = encoder.fit_transform(X[['Pclass']]).toarray()
        column_names = ["UpperClass", "MidClass", "WorkClass"]
        for i in range(len(column_names)):
            X[column_names[i]] = matrix.T[i]
        return X

In [6]:
class FeatureDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.drop(["Name", "Ticket", "Sex", "Embarked", "Pclass", "Parch", "SibSp", "Cabin", "Age", "Fare"], axis=1)

In [7]:
titanic_data = pd.read_csv('./train.csv')

In [8]:
pipeline1 = Pipeline([
    ('encoder', FeatureEncoder()),
    ('dropper', FeatureDropper())
])

In [9]:
strat_titanic_data = pipeline1.fit_transform(titanic_data)

In [10]:
strat_titanic_data["Female"] = strat_titanic_data["Female"].astype(int)
strat_titanic_data["Male"] = strat_titanic_data["Male"].astype(int)
strat_titanic_data["France"] = strat_titanic_data["France"].astype(int)
strat_titanic_data["Ireland"] = strat_titanic_data["Ireland"].astype(int)
strat_titanic_data["Southham"] = strat_titanic_data["Southham"].astype(int)
strat_titanic_data["UpperClass"] = strat_titanic_data["UpperClass"].astype(int)
strat_titanic_data["MidClass"] = strat_titanic_data["MidClass"].astype(int)
strat_titanic_data["WorkClass"] = strat_titanic_data["WorkClass"].astype(int)
strat_titanic_data.head(11)

,PassengerId,Survived,Female,Male,France,Ireland,Southham,UpperClass,MidClass,WorkClass
0,1,0,0,1,0,0,1,0,0,1
1,2,1,1,0,1,0,0,1,0,0
2,3,1,1,0,0,0,1,0,0,1
3,4,1,1,0,0,0,1,1,0,0
4,5,0,0,1,0,0,1,0,0,1
5,6,0,0,1,0,1,0,0,0,1
6,7,0,0,1,0,0,1,1,0,0
7,8,0,0,1,0,0,1,0,0,1
8,9,1,1,0,0,0,1,0,0,1
9,10,1,1,0,1,0,0,0,1,0


In [11]:
count = len(titanic_data)

In [12]:
DeckA = np.eye(count, 1, dtype=int)
DeckB = np.eye(count, 1, dtype=int) 
DeckC = np.eye(count, 1, dtype=int)
DeckD = np.eye(count, 1, dtype=int)
DeckE = np.eye(count, 1, dtype=int)
DeckF = np.eye(count, 1, dtype=int)
DeckG = np.eye(count, 1, dtype=int)
has_A = titanic_data["Cabin"].str.contains('A')
has_B = titanic_data["Cabin"].str.contains('B')
has_C = titanic_data["Cabin"].str.contains('C')
has_D = titanic_data["Cabin"].str.contains('D')
has_E = titanic_data["Cabin"].str.contains('E')
has_F = titanic_data["Cabin"].str.contains('F')
has_G = titanic_data["Cabin"].str.contains('G')

for i in range(len(strat_titanic_data)):
    if (has_A[i] == True):
        DeckA[i] = 1
    elif (has_B[i] == True):
        DeckB[i] = 1
    elif (has_C[i] == True):
        DeckC[i]=1
    elif (has_D[i] == True):
        DeckD[i] = 1
    elif (has_E[i] == True):
        DeckE[i] = 1
    elif (has_F[i] == True):
        DeckF[i] = 1
    elif (has_G[i] == True):
        DeckG[i] = 1
    else:
        DeckA[i] = 0
        DeckB[i] = 0
        DeckC[i] = 0
        DeckD[i] = 0
        DeckE[i] = 0
        DeckF[i] = 0
        DeckG[i] = 0

DeckA = pd.DataFrame(DeckA)
DeckB = pd.DataFrame(DeckB)
DeckC = pd.DataFrame(DeckC)
DeckD = pd.DataFrame(DeckD)
DeckE = pd.DataFrame(DeckE)
DeckF = pd.DataFrame(DeckF)
DeckG = pd.DataFrame(DeckG)

strat_titanic_data["DeckA"] = DeckA
strat_titanic_data["DeckB"] = DeckB
strat_titanic_data["DeckC"] = DeckC
strat_titanic_data["DeckD"] = DeckD
strat_titanic_data["DeckE"] = DeckE
strat_titanic_data["DeckF"] = DeckF
strat_titanic_data["DeckG"] = DeckG

In [13]:
Infant = np.eye(count, 1, dtype=int)
for i in range(count):
    if (titanic_data.iloc[i]["Age"] <= 5):
        Infant[i] = 1
    else:
        Infant[i] = 0
Infant = pd.DataFrame(Infant)
strat_titanic_data["Infant"] = Infant

In [14]:
Older = np.eye(count, 1, dtype=int)
VOld = np.eye(count, 1, dtype=int)
for i in range(count):
    if (titanic_data.iloc[i]["Age"] > 63) & (titanic_data.iloc[i]["Age"] < 75):
        Older[i] = 1
    elif (titanic_data.iloc[i]["Age"] > 74):
        VOld[i] = 1
    else:
        Older[i] = 0
        VOld[i] = 0
Older = pd.DataFrame(Older)
VOld = pd.DataFrame(VOld)
strat_titanic_data["Older"] = Older
strat_titanic_data["VOld"] = VOld

In [15]:
Teen = np.eye(count, 1, dtype=int)
Child = np.eye(count, 1, dtype=int)
for i in range(count):
    if (titanic_data.iloc[i]["Age"] > 12) & (titanic_data.iloc[i]["Age"] <= 18):
        Teen[i] = 1
    elif (titanic_data.iloc[i]["Age"] > 5) & (titanic_data.iloc[i]["Age"] <= 12):
        Child[i] = 1     
    else:
        Teen[i] = 0
        Child[i] = 0
Teen = pd.DataFrame(Teen)
Child = pd.DataFrame(Child)
strat_titanic_data["Teen"] = Teen
strat_titanic_data["Child"] = Child

In [16]:
Adult = np.eye(count, 1, dtype=int)
MidLife = np.eye(count, 1, dtype=int)
for i in range(count):
    if (titanic_data.iloc[i]["Age"] > 18) & (titanic_data.iloc[i]["Age"] <= 40):
        Adult[i] = 1
    elif (titanic_data.iloc[i]["Age"] > 40) & (titanic_data.iloc[i]["Age"] <= 63):
        MidLife[i] = 1
    else:
        Adult[i] = 0
        MidLife[i] = 0
Adult = pd.DataFrame(Adult)
MidLife = pd.DataFrame(MidLife)
strat_titanic_data["Adult"] = Adult
strat_titanic_data["MidLife"] = MidLife

In [17]:
Family = np.eye(count, 1, dtype=int)
for i in range(count):
    if (titanic_data.iloc[i]["Parch"] == 3):
        Family[i] = 5
    elif (titanic_data.iloc[i]["Parch"] == 1):
        Family[i] = 4
    elif (titanic_data.iloc[i]["Parch"] == 2):
        Family[i] = 3
    elif (titanic_data.iloc[i]["Parch"] == 0):
        Family[i] = 2
    elif (titanic_data.iloc[i]["Parch"] == 5):
        Family[i] = 1
    else:
        Family[i] = 0
Family = pd.DataFrame(Family)
strat_titanic_data["Family"] = Family
print(strat_titanic_data["Family"].value_counts())

Family
2    678
4    118
3     80
1      5
5      5
0      5
Name: count, dtype: int64


In [18]:
Sibling = np.eye(count, 1, dtype=int)
for i in range(count):
    if (titanic_data.iloc[i]["SibSp"] == 1):
        Sibling[i] = 5
    elif (titanic_data.iloc[i]["SibSp"] == 2):
        Sibling[i] = 4
    elif (titanic_data.iloc[i]["SibSp"] == 0):
        Sibling[i] = 3
    elif (titanic_data.iloc[i]["SibSp"] == 3):
        Sibling[i] = 2
    elif (titanic_data.iloc[i]["SibSp"] == 4):
        Sibling[i] = 1
    else:
        Sibling[i] = 0
Sibling = pd.DataFrame(Sibling)
strat_titanic_data["Sibling"] = Sibling

In [19]:
FareSHigh = np.eye(count, 1, dtype=int)
FareVHigh = np.eye(count, 1, dtype=int)
FareHigh = np.eye(count, 1, dtype=int)
FareMid = np.eye(count, 1, dtype=int)
FareLMid = np.eye(count, 1, dtype=int)
FareLow = np.eye(count, 1, dtype=int)
FareVLow = np.eye(count, 1, dtype=int)
for i in range(count):
    if (titanic_data.iloc[i]["Fare"] > 300):
        FareSHigh[i] = 1
    elif ((titanic_data.iloc[i]["Fare"] > 200) & (titanic_data.iloc[i]["Fare"] <= 300)):
        FareVHigh[i] = 1
    elif ((titanic_data.iloc[i]["Fare"] > 100) & (titanic_data.iloc[i]["Fare"] <=200)):
        FareHigh[i] = 1
    elif ((titanic_data.iloc[i]["Fare"] > 50) & (titanic_data.iloc[i]["Fare"] <=100)):
        FareMid[i] = 1
    elif ((titanic_data.iloc[i]["Fare"] > 29) & (titanic_data.iloc[i]["Fare"] <= 50)):
        FareLMid[i] = 1
    elif ((titanic_data.iloc[i]["Fare"] > 10) & (titanic_data.iloc[i]["Fare"] <= 29)):
        FareLow[i] = 1
    elif (titanic_data.iloc[i]["Fare"] <= 10):
        FareVLow[i] = 1
    else:
        FareSHigh[i] = 0
        FareVHigh[i] = 0
        FareHigh[i] = 0
        FareMid[i] = 0
        FareLMid[i] = 0
        FareLow[i] = 0
        FareVLow[i] = 0
FareSHigh = pd.DataFrame(FareSHigh)
FareVHigh = pd.DataFrame(FareVHigh)
FareHigh = pd.DataFrame(FareHigh)
FareMid = pd.DataFrame(FareMid)
FareLMid = pd.DataFrame(FareLMid)
FareLow = pd.DataFrame(FareLow)
FareVLow = pd.DataFrame(FareVLow)
strat_titanic_data["FareSHigh"] = FareSHigh
strat_titanic_data["FareVHigh"] = FareVHigh
strat_titanic_data["FareHigh"] = FareHigh
strat_titanic_data["FareMid"] = FareMid
strat_titanic_data["FareLMid"] = FareLMid
strat_titanic_data["FareLow"] = FareLow
strat_titanic_data["FareVLow"] = FareVLow

In [20]:
X = strat_titanic_data.drop(["Survived", "PassengerId", "Ireland"], axis=1)
y = strat_titanic_data['Survived']

X_data = X
y_data = y.to_numpy()
X, y = load_iris(return_X_y=True)
X_dataNB = X
y_dataNB = y

In [21]:
treeT = DecisionTreeClassifier(max_depth=10)

treeT.fit(X_data, y_data)
predict_y = treeT.predict(X_data)
predict_y = np.round(predict_y)
predict_y = predict_y.astype(int)

In [22]:
class Net(nn.Module):
    def __init__(self, name=None, input_features=30) :
        super(Net, self).__init__()
        if name:
            self.name = name
        self.fc1 = nn.Linear(input_features, 128)
        self.bn1 = nn.BatchNorm1d(128) # Batch normalization
        self.dropout1 = nn.Dropout(0.3) # Dropout for regularization

        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.3)

        self.fc4 = nn.Linear(32, 2) # Output layer for binary classification (survived/not survived)

        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x


net = Net(name='LetNet5')

LetNet5: total params: 14818


In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [24]:
X_tensor = torch.tensor(X_data.values, dtype=torch.float32)
y_tensor = torch.tensor(y_data, dtype=torch.long)

# Create a custom Dataset
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create the dataset and DataLoader
train_dataset = TitanicDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Check if CUDA is available and move the model and data to GPU if it is
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

# Training loop
epochs = 1300
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # Print every 10 mini-batches
            running_loss = 0.0

# Evaluate the trained model on the training data (optional)
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {total} training samples: {100 * correct / total:.2f} %')

Accuracy of the network on the 891 training samples: 90.57 %


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import torch
import torch.nn.functional as F

Voilà with --show_tracebacks=True

# Define the prediction function for Decision Tree
def predict_survival_tree_dropdown(b):
    with output:
        output.clear_output()
        # Create a dictionary from the input widget values, mapping dropdown selection to the corresponding one-hot encoded column
        user_data_dict = {}
        # Initialize all relevant one-hot encoded columns to 0
        for col in X_data.columns:
          user_data_dict[col] = 0

        # Set the selected gender
        if gender_dropdown.value == 'Female':
            user_data_dict['Female'] = 1
        else: # Male
            user_data_dict['Male'] = 1

        # Set the selected location
        if location_dropdown.value == 'France':
            user_data_dict['France'] = 1
        elif location_dropdown.value == 'Southampton':
            user_data_dict['Southham'] = 1
        # Ireland is dropped, so no column for Ireland

        # Set the selected social class
        if social_class_dropdown.value == 'UpperClass':
            user_data_dict['UpperClass'] = 1
        elif social_class_dropdown.value == 'MidClass':
            user_data_dict['MidClass'] = 1
        else: # WorkingClass
            user_data_dict['WorkClass'] = 1

        # Set the selected deck
        deck_map = {'A': 'DeckA', 'B': 'DeckB', 'C': 'DeckC', 'D': 'DeckD', 'E': 'DeckE', 'F': 'DeckF', 'G': 'DeckG'}
        if deck_dropdown.value in deck_map:
            user_data_dict[deck_map[deck_dropdown.value]] = 1

        # Set the selected age group
        age_group_map = {
            'Infant': 'Infant',
            'Older': 'Older',
            'VOld': 'VOld',
            'Teen': 'Teen',
            'Child': 'Child',
            'Adult': 'Adult',
            'MidLife': 'MidLife'
        }
        if age_group_dropdown.value in age_group_map:
            user_data_dict[age_group_map[age_group_dropdown.value]] = 1

        # Set the Family and Sibling values from the dropdowns (they are categorical representations)
        user_data_dict['Family'] = int(family_group_dropdown.value)
        user_data_dict['Sibling'] = int(sibling_dropdown.value)


        # Set the selected fare category
        fare_map = {
            'FareSHigh': 'FareSHigh',
            'FareVHigh': 'FareVHigh',
            'FareHigh': 'FareHigh',
            'FareMid': 'FareMid',
            'FareLMid': 'FareLMid',
            'FareLow': 'FareLow',
            'FareVLow': 'FareVLow'
        }
        if fare_dropdown.value in fare_map:
            user_data_dict[fare_map[fare_dropdown.value]] = 1


        # Convert the dictionary to a pandas DataFrame
        user_df = pd.DataFrame([user_data_dict])

        user_df = user_df.reindex(columns=X_data.columns)

        # Make sure columns are in the same order as training data
        user_df = user_df[X_data.columns]


        # Make the prediction using the trained Decision Tree model (treeT)
        prediction = treeT.predict(user_df)

        if prediction[0] == 1:
            print("Prediction of Decision Tree: Survived")
        else:
            print("Prediction of Decision Tree: Did Not Survive")

# Define the prediction function using the trained LeNet5 model (net)
def predict_survival_nn_dropdown(b):
    with output:
        output.clear_output()
        # Create a dictionary from the input widget values, mapping dropdown selection to the corresponding one-hot encoded column
        user_data_dict = {}
        # Initialize all relevant one-hot encoded columns to 0
        for col in X_data.columns:
          user_data_dict[col] = 0

        # Set the selected gender
        if gender_dropdown.value == 'Female':
            user_data_dict['Female'] = 1
        else: # Male
            user_data_dict['Male'] = 1

        # Set the selected location
        if location_dropdown.value == 'France':
            user_data_dict['France'] = 1
        elif location_dropdown.value == 'Southampton':
            user_data_dict['Southham'] = 1
        # Ireland is dropped, so no column for Ireland

        # Set the selected social class
        if social_class_dropdown.value == 'UpperClass':
            user_data_dict['UpperClass'] = 1
        elif social_class_dropdown.value == 'MidClass':
            user_data_dict['MidClass'] = 1
        else: # WorkingClass
            user_data_dict['WorkClass'] = 1

        # Set the selected deck
        deck_map = {'A': 'DeckA', 'B': 'DeckB', 'C': 'DeckC', 'D': 'DeckD', 'E': 'DeckE', 'F': 'DeckF', 'G': 'DeckG'}
        if deck_dropdown.value in deck_map:
            user_data_dict[deck_map[deck_dropdown.value]] = 1

        # Set the selected age group
        age_group_map = {
            'Infant': 'Infant',
            'Older': 'Older',
            'VOld': 'VOld',
            'Teen': 'Teen',
            'Child': 'Child',
            'Adult': 'Adult',
            'MidLife': 'MidLife'
        }
        if age_group_dropdown.value in age_group_map:
            user_data_dict[age_group_map[age_group_dropdown.value]] = 1

        # Set the Family and Sibling values from the dropdowns (they are categorical representations)
        user_data_dict['Family'] = int(family_group_dropdown.value)
        user_data_dict['Sibling'] = int(sibling_dropdown.value)

        # Set the selected fare category
        fare_map = {
            'FareSHigh': 'FareSHigh',
            'FareVHigh': 'FareVHigh',
            'FareHigh': 'FareHigh',
            'FareMid': 'FareMid',
            'FareLMid': 'FareLMid',
            'FareLow': 'FareLow',
            'FareVLow': 'FareVLow'
        }
        if fare_dropdown.value in fare_map:
            user_data_dict[fare_map[fare_dropdown.value]] = 1

        # Convert the dictionary to a pandas DataFrame
        user_df = pd.DataFrame([user_data_dict])
        user_df = user_df.reindex(columns=X_data.columns)

        # Make sure columns are in the same order as training data
        user_df = user_df[X_data.columns]


        # Convert the DataFrame to a PyTorch tensor and move to device
        user_tensor = torch.tensor(user_df.values, dtype=torch.float32).to(device)

        # Make the prediction using the trained LeNet5 model (net)
        net.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            outputs = net(user_tensor)
            _, predicted = torch.max(outputs.data, 1)

        if predicted.item() == 1:
            print("Prediction (LeNet5): Survived")
        else:
            print("Prediction (LeNet5): Did Not Survive")


# Create an output widget to display the prediction
output = widgets.Output()

# Create buttons to trigger the prediction for both models
predict_button_tree = widgets.Button(description="Predict Survival (Decision Tree)")
predict_button_nn = widgets.Button(description="Predict Survival (LeNet5)")

# Attach the prediction function to the button click events
predict_button_tree.on_click(predict_survival_tree_dropdown)
predict_button_nn.on_click(predict_survival_nn_dropdown)


# Arrange the widgets vertically for display
input_widgets = widgets.VBox([
    gender_dropdown,
    location_dropdown,
    social_class_dropdown,
    deck_dropdown,
    age_group_dropdown,
    family_group_dropdown,
    sibling_dropdown,
    fare_dropdown
])

# Display the input widgets, buttons, and output
display(input_widgets, widgets.HBox([predict_button_tree, predict_button_nn]), output)

In [ ]:
Voilà